Import Statements

In [1]:
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import numpy as np

KaggleApi Settings

In [2]:
api = KaggleApi()
api.authenticate()

Import Datasets with Kaggle.API

In [3]:
api.dataset_download_file('thedevastator/global-fossil-co2-emissions-by-country-2002-2022',
                          file_name='GCB2022v27_MtCO2_flat.csv')
with zipfile.ZipFile('GCB2022v27_MtCO2_flat.csv.zip', 'r') as zipref:
    zipref.extractall()

In [4]:
api.dataset_download_file('russellyates88/suicide-rates-overview-1985-to-2016',
                          file_name='master.csv')
with zipfile.ZipFile('master.csv.zip', 'r') as zipref:
    zipref.extractall()

In [5]:
api.dataset_download_file('yapwh1208/countries-gdp-2012-to-2021',
                          file_name='GDP.csv')

False

In [6]:
api.dataset_download_file('folaraz/world-countries-and-continents-details',
                          file_name='Countries Longitude and Latitude.csv')

False

Read and Filter DataFiles

In [7]:
df_emissions = pd.read_csv("GCB2022v27_MtCO2_flat.csv", sep=",")

new_df_emissions = df_emissions[(df_emissions['Year'] >= 2006) & (df_emissions['Year'] <= 2016)]
new_df_emissions.rename(columns={'Year':'year', 'Country':'country'}, inplace=True)
new_df_emissions = new_df_emissions.drop(['Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Other', 'Per Capita'], axis=1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_26932\2350370852.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_emissions.rename(columns={'Year':'year', 'Country':'country'}, inplace=True)


In [8]:
df_suicide = pd.read_csv("master.csv", sep=",")

new_df_suicide = df_suicide[(df_suicide['year'] >= 2006) & (df_suicide['year'] <= 2016)]
new_df_suicide = new_df_suicide.groupby(['country', 'year']).sum(). reset_index()
new_df_suicide = new_df_suicide.drop(['HDI for year', 'gdp_per_capita ($)'], axis=1)

In [9]:
df_gdp = pd.read_csv("GDP.csv", sep=",")

new_df_gdp = df_gdp[['Country Name', 'Country Code', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']]
new_df_gdp = new_df_gdp.melt(id_vars=["Country Name", "Country Code"], var_name="year")
new_df_gdp['year'] = new_df_gdp['year'].astype('int64')
new_df_gdp.rename(columns={'Country Name':'country', 'value':'GDP_Value'}, inplace=True)

In [10]:
df_allCountries = pd.read_csv("Countries%20Longitude%20and%20Latitude.csv", sep=",")
df_allCountries.rename(columns={'name':'country'}, inplace=True)


Conditional statements, loop control statements and loops

In [21]:
df_countries_EUROPE = pd.read_csv(r'Country_ISOCode.csv', delimiter=',')
df_countries_EUROPE

# Create an empty DataFrame with the same columns as 'new_df'
european_df_newDfEmission = pd.DataFrame(columns=new_df_emissions.columns)

# Iterate over the rows of the 'new_df' dataframe
for index, row in new_df_emissions.iterrows():
    # Get the country from the current row
    iso = row['ISO 3166-1 alpha-3']
    
    # Check if the country is in the list of European countries
    if iso in df_countries_EUROPE['ISO Code'].values:
        # Append the row to the 'european_df'
        european_df_newDfEmission = pd.concat([european_df_newDfEmission, row.to_frame().T])
        
# Reset the index of the new dataframe
european_df_newDfEmission .reset_index(drop=True, inplace=True)

# Print the new dataframe with the European countries
european_df_newDfEmission 

# Count the occurrences of each country
country_counts = european_df_newDfEmission['country'].value_counts()

# Print the counts
print(country_counts)


country
Albania                   11
Romania                   11
North Macedonia           11
Malta                     11
Montenegro                11
Netherlands               11
Norway                    11
Poland                    11
Portugal                  11
Moldova                   11
Russia                    11
Lithuania                 11
Serbia                    11
Slovakia                  11
Slovenia                  11
Spain                     11
Sweden                    11
Switzerland               11
Turkey                    11
Ukraine                   11
Luxembourg                11
Liechtenstein             11
Andorra                   11
Estonia                   11
Austria                   11
Belarus                   11
Belgium                   11
Bosnia and Herzegovina    11
Bulgaria                  11
Croatia                   11
Czech Republic            11
Denmark                   11
Finland                   11
Latvia                    11
France

kombinieren


In [12]:
merged_df_emissions_suicide = pd.merge(new_df_emissions, new_df_suicide, on=['country', 'year'])
merged_df_emissions_suicide = pd.merge(merged_df_emissions_suicide, df_allCountries, on=['country'])
merged_df_emissions_suicide = merged_df_emissions_suicide.drop('Unnamed: 0', axis=1)


In [13]:
merged_df_emissions_gdp = pd.merge(new_df_emissions, new_df_gdp, on=['country', 'year'])
merged_df_emissions_gdp = merged_df_emissions_gdp.drop(['Country Code'], axis=1)
merged_df_emissions_gdp = pd.merge(merged_df_emissions_gdp, df_allCountries, on=['country'])
merged_df_emissions_gdp = merged_df_emissions_gdp.drop('Unnamed: 0', axis=1)


In [14]:
merged_df_suicide_gdp = pd.merge(new_df_suicide, new_df_gdp, on=['country', 'year'])
merged_df_suicide_gdp = pd.merge(merged_df_suicide_gdp, df_allCountries, on=['country'])
merged_df_suicide_gdp = merged_df_suicide_gdp.drop('Unnamed: 0', axis=1)

Use of Python built-in data structures (lists, dictionaries, sets, tuples)

In [15]:
merged_df_emissions_suicide = merged_df_emissions_suicide.fillna(0)
merged_df_emissions_gdp = merged_df_emissions_gdp.fillna(0)
merged_df_suicide_gdp = merged_df_suicide_gdp.fillna(0)
df_countries = df_countries.fillna(0)
df_allCountries = df_allCountries.fillna(0)

Use of tables, vizualizations/graphics for data exploration

Integration of a statistical analyses (correlation analysis, statical tests)

In [18]:
#merged_df_emissions_gdp.to_csv(r'final_csv/merged_df_emissions_gdp.csv', index=False, header=True)
#merged_df_suicide_gdp.to_csv(r'final_csv/merged_df_suicide_gdp.csv', index=False, header=True)
#merged_df_emissions_suicide.to_csv(r'final_csv/merged_df_emissions_suicide.csv', index=False, header=True)